It is a Grid Search based SLAM implementation using LIDAR data using the following:
 - basic : we rotate the destination (later) pcd to destination's (local) grid points to score against source (former) pcd
 - adaptive grid : grid is interpolated based on velocity instead of distance ; this assumes that there are no sudden acceleration or impulsive forces or jerks
 - 2d map : creates a 2d map of the environment instead of 3d
 - parallel : the grid search for every pose has been parallelized on CPU cores

### Imports

In [1]:
import numpy as np
import pandas as pd
import gc
from tqdm import *
import os
import copy
import sys
import time
from collections import Counter, deque
from multiprocessing import Pool, cpu_count

import transforms3d

from sklearn.manifold import TSNE, LocallyLinearEmbedding
from sklearn.cluster import MeanShift

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv
import open3d as o3d

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
# %matplotlib notebook

### Set paths and file names

In [2]:
# ATM_PATH = "/Users/saby/Projects/atm/"
ATM_PATH = "/home/nilesh/Prashant/sabyasachi/atm"
DATA_PATH = os.path.join(ATM_PATH, "data", "data", "datasets")

IISC_DATA_PATH = os.path.join(DATA_PATH, "IISC")
# EXP_PATH = os.path.join(IISC_DATA_PATH, "2019-02-26","ref_run_ccw_r")
EXP_PATH = os.path.join(IISC_DATA_PATH, "2019-03-30","15-41-43-mule1-auto-slam2d_test_track_ccw_5loops")

LIDAR_PCD_PATH = os.path.join(EXP_PATH, "complete")
LIDAR_CSV_PATH = os.path.join(IISC_DATA_PATH, "lidar.csv")

MAP_FILE = 'map_5loops.pcd'
POSE_FILE = 'pose_5loops.json'

### Set constants and flags

In [3]:
# FIRST_PCD = 200
# FINAL_PCD = 210
# FIRST_PCD = 2
# FINAL_PCD = 805

FIRST_PCD = 2
FINAL_PCD = 4694

VOXEL_SZ = 0.2
MAKE_2D = False
APPLY_CLUSTER = False

plt.rcParams['figure.figsize'] = [10, 10]

X_GRID = np.arange(-0.2, 0.225, 0.025)
Y_GRID = np.arange(-0.2, 0.225, 0.025)
THETA_GRID = np.arange(-2, 2.25, 0.25)
print("No of grid pts to evaluate per pose: {}".format(X_GRID.shape[0]*Y_GRID.shape[0]*THETA_GRID.shape[0]))

No of grid pts to evaluate per pose: 4913


### Functions for pcd io

In [4]:
def pose2matrix(translation_list, rotation_angle_list):
    trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom = np.ones(3)
    transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom)
    return transform_mat

def filter_pcd(old_pcd,
               apply_downsample = True,
               downsample_voxel_size = VOXEL_SZ,
               
               apply_outlier_removal = True,
               downsample_radius = 1,
               downsample_neighbors = 20,
               
               apply_crop = True,
               crop_min_arr = np.array([-100,-100,0]),
               crop_max_arr = np.array([100,100,100]),
               
               apply_cluster = True,
               cluster_neighbours = 30,
               cluster_labels = 2):
    np.random.seed(0)
    pcd = copy.deepcopy(old_pcd)
    
    if apply_downsample:
        voxel_down_pcd = o3d.geometry.voxel_down_sample(pcd, voxel_size = downsample_voxel_size)
        pcd = voxel_down_pcd

    if apply_outlier_removal:
        denser_pcd, ind = o3d.geometry.radius_outlier_removal(pcd,
                                                              nb_points = downsample_neighbors,
                                                              radius    = downsample_radius)
        pcd = denser_pcd
    
    if apply_crop:
        cropped_pcd = o3d.geometry.crop_point_cloud(pcd, crop_min_arr, crop_max_arr)
        pcd = cropped_pcd

    if apply_cluster:
        few_pts = np.asarray(pcd.points)
        try:
            few_pts_reduced = LocallyLinearEmbedding(n_neighbors=cluster_neighbours, n_components=2).fit_transform(few_pts)
        except Exception as e:
            try:
                few_pts_reduced = LocallyLinearEmbedding(n_neighbors=cluster_neighbours, n_components=2, eigen_solver='dense').fit_transform(few_pts)
            except Exception as e:
                few_pts_reduced = few_pts
        clf = MeanShift().fit(few_pts_reduced)
        pcd.points = o3d.utility.Vector3dVector(few_pts[clf.labels_ < cluster_labels])
    
    return pcd

def make_2d(pcd):
    new_pcd = copy.deepcopy(pcd)
    new_pts = np.concatenate([np.asarray(pcd.points)[:,:-1],np.zeros((len(pcd.points),1))], axis=1)
    new_pcd.points = o3d.utility.Vector3dVector(new_pts)
    return new_pcd

def read_pcd(pcd_id):
    pcd_file = str(pcd_id) + ".pcd"
    pcd = o3d.io.read_point_cloud(os.path.join(LIDAR_PCD_PATH, pcd_file))
    return pcd

def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.show()
    elif where is 'mat_2d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.show()
        
def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

### Functions for pose estimation and scoring

In [5]:
def score_pts(src_pcd, dst_pcd, trans_arr=np.zeros(3), rot_ang=np.zeros(3)):
    NEIGHB_RADIUS = VOXEL_SZ
    
    src_tmp_pcd = copy.deepcopy(src_pcd)
    dst_tmp_pcd = copy.deepcopy(dst_pcd)
    
    transform_mat = pose2matrix(trans_arr, rot_ang)
    # Note: we apply transformation on target pcd to directly get pose (without sign change)
    dst_tmp_pcd.transform(transform_mat)
    
    score = o3d.registration.evaluate_registration(src_tmp_pcd, dst_tmp_pcd, NEIGHB_RADIUS)
    
    fit = score.fitness
    rmse = score.inlier_rmse
    n_pairs = np.asarray(score.correspondence_set).shape[0]
    return rmse, fit, n_pairs

def parallel_thread(parallel_arg):
    grid_pts, previous_pose, previous_velocity = parallel_arg[0], parallel_arg[1], parallel_arg[2]
    x_pt, y_pt, theta_pt = grid_pts[0], grid_pts[1], grid_pts[2]
    x_src, y_src, theta_src = previous_pose[0], previous_pose[1], previous_pose[2]
    x_vel, y_vel, theta_vel = previous_velocity[0], previous_velocity[1], previous_velocity[2]
    
    x = x_src + x_vel + x_pt
    y = y_src + y_vel + y_pt
    t = theta_src + theta_vel + theta_pt
    rmse, fit, n_pairs = score_pts(map_pcd, new_pcd,
                                   trans_arr = [x, y, 0],
                                   rot_ang   = [0, 0, t])
    result = {'x' : x,
              'y' : y,
              'theta' : t,
              
              'x_vel' : x_vel + x_pt,
              'y_vel' : y_vel + y_pt,
              'theta_vel' : theta_vel + theta_pt,
              
              'rmse' : rmse,
              'fit' : fit,
              'n_pairs' : n_pairs}
#     print(grid_pts)
    return result

### Implementation

#### Create initial map

In [6]:
map_pcd = read_pcd(FIRST_PCD)
map_pcd = filter_pcd(map_pcd, apply_cluster=APPLY_CLUSTER)
if MAKE_2D:
    map_pcd = make_2d(map_pcd)
print(map_pcd)

geometry::PointCloud with 3621 points.


#### Initialize pose arrays

In [7]:
pose_list = []
global_origin_pt = {'x' : 0,
                    'y' : 0,
                    'theta' : 0,
                    
                    'x_vel' : 0,
                    'y_vel' : 0,
                    'theta_vel' : 0,
                    
                    'rmse' : 0,
                    'fit' : 1,
                    'n_pairs' : np.asarray(map_pcd.points).shape[0]}
pose_list.append(global_origin_pt)

#### Iterate over all subsequent pcds for local pose estimation and building the map

In [ ]:
for pcd_idx in tqdm_notebook(range(FIRST_PCD+1, FINAL_PCD)):
    # Read next pcd and filter it
    new_pcd = read_pcd(pcd_idx)
    new_pcd = filter_pcd(new_pcd, apply_cluster=APPLY_CLUSTER)
    if MAKE_2D:
        new_pcd = make_2d(new_pcd)

    # Extract last pose
    x_src, y_src, theta_src = pose_list[-1]['x'], pose_list[-1]['y'], pose_list[-1]['theta']
    x_vel, y_vel, theta_vel = pose_list[-1]['x_vel'], pose_list[-1]['y_vel'], pose_list[-1]['theta_vel']
    
    # Extract all grid pts
    parallel_args = [([x_pt, y_pt, theta], [x_src, y_src, theta_src], [x_vel, y_vel, theta_vel])\
                     for x_pt in X_GRID for y_pt in Y_GRID for theta in THETA_GRID]
    
    # Score on all grid pts
#     process_pool = Pool(cpu_count()-1)
#     score_list = [each for each in process_pool.imap_unordered(parallel_thread, parallel_args)]
#     process_pool.terminate()
    
    score_list = [parallel_thread(each) for each in parallel_args]
    
    # Extract pose with best matching score
    df_score = pd.DataFrame(score_list)
    new_pose_pt = df_score.iloc[df_score.fit.argmax()].to_dict()
    pose_list.append(new_pose_pt)

    # Transform this pcd based on the best pose
    new_tmp_pcd = copy.deepcopy(new_pcd)
    transform_mat = pose2matrix([new_pose_pt['x'], new_pose_pt['y'], 0],
                                 [0, 0, new_pose_pt['theta']])
    new_tmp_pcd.transform(transform_mat)

    # Add the current transformed pcd to the map
    map_pcd.points = o3d.utility.Vector3dVector(np.concatenate((np.asarray(map_pcd.points),
                                                                 np.asarray(new_tmp_pcd.points)), axis=0))

    map_pcd = filter_pcd(map_pcd, apply_cluster=False)
    if MAKE_2D:
        map_pcd = make_2d(map_pcd)
    
    # Checkpoint the new map and pose array to files
    o3d.io.write_point_cloud(MAP_FILE, map_pcd)
    pd.DataFrame(pose_list).to_json(POSE_FILE, orient='records', lines=True)
    if pcd_idx % 500 == 0:
        o3d.io.write_point_cloud(MAP_FILE[:-4] + str(pcd_idx) + ".pcd", map_pcd)
    
    gc.collect()
    
    print(map_pcd)

geometry::PointCloud with 3219 points.
geometry::PointCloud with 4043 points.
geometry::PointCloud with 4062 points.
geometry::PointCloud with 4040 points.
geometry::PointCloud with 4129 points.
geometry::PointCloud with 4275 points.
geometry::PointCloud with 4406 points.
geometry::PointCloud with 4321 points.
geometry::PointCloud with 4368 points.
geometry::PointCloud with 4274 points.
geometry::PointCloud with 4279 points.
geometry::PointCloud with 4643 points.
geometry::PointCloud with 4508 points.
geometry::PointCloud with 4541 points.
geometry::PointCloud with 4003 points.
geometry::PointCloud with 4559 points.
geometry::PointCloud with 4282 points.
geometry::PointCloud with 4099 points.
geometry::PointCloud with 4039 points.
geometry::PointCloud with 4014 points.
geometry::PointCloud with 4046 points.
geometry::PointCloud with 3936 points.
geometry::PointCloud with 3857 points.
geometry::PointCloud with 3892 points.
geometry::PointCloud with 3892 points.
geometry::PointCloud with

geometry::PointCloud with 12249 points.
geometry::PointCloud with 12393 points.
geometry::PointCloud with 12448 points.
geometry::PointCloud with 12484 points.
geometry::PointCloud with 12470 points.
geometry::PointCloud with 12566 points.
geometry::PointCloud with 12581 points.
geometry::PointCloud with 12589 points.
geometry::PointCloud with 12625 points.
geometry::PointCloud with 12676 points.
geometry::PointCloud with 12633 points.
geometry::PointCloud with 12652 points.
geometry::PointCloud with 12678 points.
geometry::PointCloud with 12748 points.
geometry::PointCloud with 12761 points.
geometry::PointCloud with 12802 points.
geometry::PointCloud with 12843 points.
geometry::PointCloud with 12863 points.
geometry::PointCloud with 12902 points.
geometry::PointCloud with 12829 points.
geometry::PointCloud with 12877 points.
geometry::PointCloud with 11826 points.
geometry::PointCloud with 11880 points.
geometry::PointCloud with 11965 points.
geometry::PointCloud with 12077 points.


geometry::PointCloud with 17845 points.
geometry::PointCloud with 17897 points.
geometry::PointCloud with 17946 points.
geometry::PointCloud with 17975 points.
geometry::PointCloud with 18025 points.
geometry::PointCloud with 18066 points.
geometry::PointCloud with 18116 points.
geometry::PointCloud with 18149 points.
geometry::PointCloud with 18190 points.
geometry::PointCloud with 18212 points.
geometry::PointCloud with 18244 points.
geometry::PointCloud with 18265 points.
geometry::PointCloud with 18293 points.
geometry::PointCloud with 18322 points.
geometry::PointCloud with 18351 points.
geometry::PointCloud with 18367 points.
geometry::PointCloud with 18383 points.
geometry::PointCloud with 18396 points.
geometry::PointCloud with 18410 points.
geometry::PointCloud with 18415 points.
geometry::PointCloud with 18428 points.
geometry::PointCloud with 18437 points.
geometry::PointCloud with 18444 points.
geometry::PointCloud with 17071 points.
geometry::PointCloud with 17129 points.


geometry::PointCloud with 15417 points.
geometry::PointCloud with 15131 points.
geometry::PointCloud with 15198 points.
geometry::PointCloud with 14996 points.
geometry::PointCloud with 15022 points.
geometry::PointCloud with 15209 points.
geometry::PointCloud with 15250 points.
geometry::PointCloud with 15325 points.
geometry::PointCloud with 15448 points.
geometry::PointCloud with 15459 points.
geometry::PointCloud with 15618 points.
geometry::PointCloud with 15542 points.
geometry::PointCloud with 15787 points.
geometry::PointCloud with 15939 points.
geometry::PointCloud with 15825 points.
geometry::PointCloud with 15904 points.
geometry::PointCloud with 16102 points.
geometry::PointCloud with 16154 points.
geometry::PointCloud with 16063 points.
geometry::PointCloud with 16134 points.
geometry::PointCloud with 16183 points.
geometry::PointCloud with 16191 points.
geometry::PointCloud with 16051 points.
geometry::PointCloud with 16146 points.
geometry::PointCloud with 16035 points.


geometry::PointCloud with 18078 points.
geometry::PointCloud with 18213 points.
geometry::PointCloud with 18124 points.
geometry::PointCloud with 18057 points.
geometry::PointCloud with 18199 points.
geometry::PointCloud with 18105 points.
geometry::PointCloud with 18276 points.
geometry::PointCloud with 18367 points.
geometry::PointCloud with 18551 points.
geometry::PointCloud with 18370 points.
geometry::PointCloud with 18589 points.
geometry::PointCloud with 18753 points.
geometry::PointCloud with 18413 points.
geometry::PointCloud with 18406 points.
geometry::PointCloud with 18418 points.
geometry::PointCloud with 18344 points.
geometry::PointCloud with 18398 points.
geometry::PointCloud with 18315 points.
geometry::PointCloud with 18402 points.
geometry::PointCloud with 18636 points.
geometry::PointCloud with 18688 points.
geometry::PointCloud with 18472 points.
geometry::PointCloud with 18504 points.
geometry::PointCloud with 18617 points.
geometry::PointCloud with 18713 points.


...

geometry::PointCloud with 42655 points.
geometry::PointCloud with 42667 points.
geometry::PointCloud with 42681 points.
geometry::PointCloud with 42693 points.
geometry::PointCloud with 42699 points.
geometry::PointCloud with 42707 points.
geometry::PointCloud with 42712 points.
geometry::PointCloud with 42722 points.
geometry::PointCloud with 42728 points.
geometry::PointCloud with 42736 points.
geometry::PointCloud with 42750 points.
geometry::PointCloud with 42761 points.
geometry::PointCloud with 42770 points.
geometry::PointCloud with 42780 points.
geometry::PointCloud with 42787 points.
geometry::PointCloud with 42794 points.
geometry::PointCloud with 42800 points.
geometry::PointCloud with 42812 points.
geometry::PointCloud with 42815 points.
geometry::PointCloud with 42821 points.
geometry::PointCloud with 42826 points.
geometry::PointCloud with 42828 points.
geometry::PointCloud with 42831 points.
geometry::PointCloud with 42838 points.
geometry::PointCloud with 42841 points.


geometry::PointCloud with 43848 points.
geometry::PointCloud with 43859 points.
geometry::PointCloud with 43859 points.
geometry::PointCloud with 43867 points.
geometry::PointCloud with 43878 points.
geometry::PointCloud with 43888 points.
geometry::PointCloud with 43896 points.
geometry::PointCloud with 43904 points.
geometry::PointCloud with 43912 points.
geometry::PointCloud with 43924 points.
geometry::PointCloud with 43941 points.
geometry::PointCloud with 43956 points.
geometry::PointCloud with 43971 points.
geometry::PointCloud with 43983 points.
geometry::PointCloud with 43990 points.
geometry::PointCloud with 44021 points.
geometry::PointCloud with 44062 points.
geometry::PointCloud with 41846 points.
geometry::PointCloud with 41929 points.
geometry::PointCloud with 41980 points.
geometry::PointCloud with 42020 points.
geometry::PointCloud with 42107 points.
geometry::PointCloud with 42141 points.
geometry::PointCloud with 42192 points.
geometry::PointCloud with 42212 points.


geometry::PointCloud with 43734 points.
geometry::PointCloud with 43771 points.
geometry::PointCloud with 43792 points.
geometry::PointCloud with 43816 points.
geometry::PointCloud with 43829 points.
geometry::PointCloud with 43837 points.
geometry::PointCloud with 43854 points.
geometry::PointCloud with 43863 points.
geometry::PointCloud with 43870 points.
geometry::PointCloud with 43875 points.
geometry::PointCloud with 43883 points.
geometry::PointCloud with 43888 points.
geometry::PointCloud with 43898 points.
geometry::PointCloud with 43913 points.
geometry::PointCloud with 43934 points.
geometry::PointCloud with 43987 points.
geometry::PointCloud with 44013 points.
geometry::PointCloud with 44036 points.
geometry::PointCloud with 44090 points.
geometry::PointCloud with 44102 points.
geometry::PointCloud with 44127 points.
geometry::PointCloud with 44149 points.
geometry::PointCloud with 44157 points.
geometry::PointCloud with 44170 points.
geometry::PointCloud with 44185 points.


geometry::PointCloud with 45400 points.
geometry::PointCloud with 45408 points.
geometry::PointCloud with 45410 points.
geometry::PointCloud with 45419 points.
geometry::PointCloud with 45424 points.
geometry::PointCloud with 45430 points.
geometry::PointCloud with 45436 points.
geometry::PointCloud with 45438 points.
geometry::PointCloud with 45441 points.
geometry::PointCloud with 45450 points.
geometry::PointCloud with 45454 points.
geometry::PointCloud with 45461 points.
geometry::PointCloud with 45467 points.
geometry::PointCloud with 45481 points.
geometry::PointCloud with 45487 points.
geometry::PointCloud with 45495 points.
geometry::PointCloud with 45499 points.
geometry::PointCloud with 45508 points.
geometry::PointCloud with 45518 points.
geometry::PointCloud with 45521 points.
geometry::PointCloud with 45536 points.
geometry::PointCloud with 45542 points.
geometry::PointCloud with 45548 points.
geometry::PointCloud with 45557 points.
geometry::PointCloud with 45580 points.


geometry::PointCloud with 46468 points.
geometry::PointCloud with 46469 points.
geometry::PointCloud with 46473 points.
geometry::PointCloud with 46484 points.
geometry::PointCloud with 46510 points.
geometry::PointCloud with 46517 points.
geometry::PointCloud with 46520 points.
geometry::PointCloud with 46530 points.
geometry::PointCloud with 46536 points.
geometry::PointCloud with 46541 points.
geometry::PointCloud with 46545 points.
geometry::PointCloud with 46547 points.
geometry::PointCloud with 46550 points.
geometry::PointCloud with 46551 points.
geometry::PointCloud with 46553 points.
geometry::PointCloud with 46560 points.
geometry::PointCloud with 46585 points.
geometry::PointCloud with 46601 points.
geometry::PointCloud with 46614 points.
geometry::PointCloud with 46619 points.
geometry::PointCloud with 46623 points.
geometry::PointCloud with 46624 points.
geometry::PointCloud with 46624 points.
geometry::PointCloud with 46624 points.
geometry::PointCloud with 46625 points.


#### Verifying completing of SLAM iterations

In [ ]:
pcd_idx

In [ ]:
map_pcd

### Analysis

#### Look at the Map

In [ ]:
map_pcd

In [ ]:
draw_pcd(map_pcd, where='mat_3d')
draw_pcd(map_pcd, where='mat_2d')

In [ ]:
draw_pcd(map_pcd, where='opn_nb')

#### Look at the Poses

In [ ]:
df_pose = pd.DataFrame(pose_list)

In [ ]:
plt.scatter(df_pose['x'], df_pose['y'])

#### Did we hit the velocity boundary ?

In [ ]:
df_pose['x_abs'] = [0] + [df_pose.iloc[idx]['x_vel']- df_pose.iloc[idx-1]['x_vel'] for idx in range(df_pose.shape[0]) if idx != 0]
df_pose['y_abs'] = [0] + [df_pose.iloc[idx]['y_vel']- df_pose.iloc[idx-1]['y_vel'] for idx in range(df_pose.shape[0]) if idx != 0]
df_pose['theta_abs'] = [0] + [df_pose.iloc[idx]['theta_vel']- df_pose.iloc[idx-1]['theta_vel'] for idx in range(df_pose.shape[0]) if idx != 0]

In [ ]:
if df_pose['x_abs'].min() <= X_GRID[0]:
    print("Velocity boundary hit in -X direction")
    print("Expected: {} | Real : {}".format(X_GRID[0], df_pose['x_abs'].min()))

if df_pose['y_abs'].min() <= Y_GRID[0]:
    print("Velocity boundary hit in -Y direction")
    print("Expected: {} | Real : {}".format(Y_GRID[0], df_pose['y_abs'].min()))
    
if df_pose['theta_abs'].min() <= THETA_GRID[0]:
    print("Velocity boundary hit in -THETA direction")
    print("Expected: {} | Real : {}".format(THETA_GRID[0], df_pose['theta_abs'].min()))
    
if df_pose['x_abs'].max() >= X_GRID[-1]:
    print("Velocity boundary hit in +X direction")
    print("Expected: {} | Real : {}".format(X_GRID[-1], df_pose['x_abs'].max()))
    
if df_pose['y_abs'].max() >= Y_GRID[-1]:
    print("Velocity boundary hit in +Y direction")
    print("Expected: {} | Real : {}".format(Y_GRID[-1], df_pose['y_abs'].max()))

if df_pose['theta_abs'].max() >= THETA_GRID[-1]:
    print("Velocity boundary hit in +THETA direction")
    print("Expected: {} | Real : {}".format(THETA_GRID[-1], df_pose['theta_abs'].max()))
    
print("All clear?")